# Conservation area overlaps report
**Author**:  Greg Slater <br>
**Date**:  6th August 2024 <br>
**Dataset Scope**: conservation-area <br>
**Report Type**: Ad-hoc <br>

## Purpose
This report helps investigate issues where LPA conservation area entities are overlapping with Historic England conservation area entities, by showing issues on a map.

Run the whole notebook and use the drop-down in the bottom cell to select an LPA. By default all conservation area entities within the LPA will be displayed, but there is a toggle button which can be used to show only those with overlap issues.

Overlap issues are defined as where there is some overlap between the polygons of an LPA entity and a Historic England entity.

The categories below are used to describe the types of matches which can occur between two entities (note that a single entity can have multiple overlap issues if it overlaps with more than one other entity). These definitions use some thresholds of the % of overlap between two entities to define "match", "partial", or "edge" overlaps. The default thresholds used are highlighted in the definitions below, but they can also be changed by the user if required (see the "Geographical duplicate identification" section).


* **No overlap issues**: the two entities do not overlap at all. <br></br>
* **Complete match (two-way)**: the two entities both overlap each other by more than the match threshold (default 90%) - this suggests the boundaries of each polygon closely match. These entities can usually be merged. <br></br>
* **Single match (one-way)**: just one entity is overlapped by the other by more than the match threshold (default 90%) - this is usually where one entity is completely or partially covered by another, larger entity. These may need further investigation or escalation with the LPA. <br></br>
* **Partial match**: the two entities overlap but by less than the match treshold for each (default 90%). These may need further investigation or escalation with the LPA. <br></br>
* **Edge overlap**: the two entities both overlap each other by an amount between the edge threshold (default between 1-10%). These may not be a serious issue, but may still be raised with the LPA. <br></br>

The image below gives a visual breakdown of these categories:

<img src="overlap_issue_types.png" style="height:1.in"/> 

In [ ]:
# if running on Colab, uncomment and run this line below too:
# !pip install mapclassify

In [ ]:
import pandas as pd
import geopandas as gpd
import shapely.wkt
import urllib
import numpy as np
import os
import folium
import ipywidgets as widgets
from datetime import datetime

td = datetime.today().strftime('%Y-%m-%d')

output_dir = "../../data/reports/conservation-area-duplicates/"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def get_all_organisations():
    params = urllib.parse.urlencode({
        "sql": f"""
        select entity as organisation_entity, name as organisation_name, organisation, dataset, local_planning_authority
        from organisation
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/digital-land.csv?{params}"
    df = pd.read_csv(url)
    return df

def get_pdp_geo_dataset(dataset, underscore_cols=True, crs_out=27700):

    url = f"https://files.planning.data.gov.uk/dataset/{dataset}.geojson"
    gdf = gpd.read_file(url)

    if underscore_cols:
        gdf.columns = [x.replace("-", "_") for x in gdf.columns]


    gdf.set_crs(epsg=4326, inplace=True)
    gdf.to_crs(epsg=crs_out, inplace=True)

    return gdf

def get_provisions():
    global provisions_df  
    params = urllib.parse.urlencode({
        "sql": f"""
            SELECT
                cohort, notes, organisation, project, provision_reason, start_date
            FROM
                provision   
            WHERE 
                provision_reason = "expected"
                AND project = "open-digital-planning"
            GROUP BY organisation
            ORDER BY cohort
        """,
        "_size": "max"
    })
    url = f"https://datasette.planning.data.gov.uk/digital-land.csv?{params}"
    provisions_df = pd.read_csv(url)
    return provisions_df

def get_old_entity(collection_name):
    params = urllib.parse.urlencode({
        "sql": f"""
        select *
        from old_entity
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/{collection_name}.csv?{params}"
    df = pd.read_csv(url)
    return df

In [ ]:
def display_lpa_issues(lpa_name, show_only_issues = False):

    if show_only_issues:
        map_gdf = lpa_ca_all[
            (lpa_ca_all["lpa_name"] == lpa_name) &
            lpa_ca_all["has_overlap_issues"]].copy()
        
    else: map_gdf = lpa_ca_all[lpa_ca_all["lpa_name"] == lpa_name].copy()

    print(map_gdf.groupby(["lpa_name", "overlap_issue_types"], dropna=False).size().reset_index(name = "n_entities"))

    ents_lpa = map_gdf[map_gdf["organisation_entity"] != 16]
    ents_he = map_gdf[map_gdf["organisation_entity"] == 16]

    if (len(ents_lpa) == 0) | (len(ents_he) == 0):
        print("")
        print("No entities for one of the organisations, not possible to display map")
        return None
    
    else:

        m = ents_lpa.explore(
            tiles = "CartoDB positron",
            column = "organisation_name", 
            cmap = "Accent",
            highlight = True,
            tooltip = ["organisation_name", "entity", "reference", "name", "has_overlap_issues", "n_overlap_issues", "overlap_issue_types"],
            style_kwds={"fillOpacity" : "0.1"},
            name = "LPA entities")

        ents_he.explore(
            m = m,
            column = "organisation_name", 
            cmap = ["#bf5b16"],
            highlight = True,
            tooltip = ["organisation_name", "entity", "reference", "name", "has_overlap_issues", "n_overlap_issues", "overlap_issue_types"],
            style_kwds={"fillOpacity" : "0.1"},
            name = "Historic England entities")

        folium.LayerControl(show = True).add_to(m)  # use folium to add layer control

        return m


## Data in

In [ ]:
# get orgs and flag ODP
provisions_df = get_provisions()

org_df = get_all_organisations()
org_df["odp_flag"] = np.where(org_df["organisation"].isin(provisions_df["organisation"]), True, False)
print(len(org_df))

In [ ]:
# CA from pdp
ca_in = pd.read_csv("https://files.planning.data.gov.uk/dataset/conservation-area.csv",
                            usecols = ["entity", "name", "organisation-entity", "reference", "entry-date", "point", "geometry"])

ca_in.columns = [x.replace("-", "_") for x in ca_in.columns]

ca_df = ca_in[ca_in["geometry"].notnull()].copy()

# join organisation name and LPA codes from lookup
ca_df = ca_df.merge(
    org_df[["organisation_entity", "organisation_name"]], 
    how = "left",
    on = "organisation_entity")

# load to gdf, both point and poly versions
ca_df["point"] = ca_df["point"].apply(shapely.wkt.loads)
ca_point_gdf = gpd.GeoDataFrame(ca_df, geometry="point")

ca_df["geometry"] = ca_df["geometry"].apply(shapely.wkt.loads)
ca_poly_gdf = gpd.GeoDataFrame(ca_df, geometry="geometry")

# Transform to ESPG:27700 for more interpretable area units
ca_point_gdf.set_crs(epsg=4326, inplace=True)
ca_point_gdf.to_crs(epsg=27700, inplace=True)

ca_poly_gdf.set_crs(epsg=4326, inplace=True)
ca_poly_gdf.to_crs(epsg=27700, inplace=True)

print(len(ca_poly_gdf))

In [ ]:
# LPA boundaries from PDP site
lpa_gdf = get_pdp_geo_dataset("local-planning-authority")
lpa_gdf.rename(columns={'name':'lpa_name', 'reference':'lpa_reference'}, inplace=True)
lpa_gdf["ODP_flag"] = np.where(lpa_gdf["lpa_reference"].isin(org_df[org_df["odp_flag"]]["local_planning_authority"]), True, False)
print(len(lpa_gdf))

## Calculations
### Spatial joining - LPA boundaries to conservation area points

In [ ]:
# join LPAs to all conservation areas, then join on the names of supplying organisations for matching conservation areas
lpa_ca_join = gpd.sjoin(
    lpa_gdf[["lpa_reference", "lpa_name", "ODP_flag", "geometry"]],
    ca_point_gdf[["entity", "organisation_entity", "organisation_name", "point"]],
    how = "left",
    predicate = "intersects"
)

print(len(lpa_ca_join))


### Geographical duplicate identification

In [ ]:
MATCH_LOWER_THRESH = 0.95  # defines the lower limit of the shared overlap between two entities to be called a match
EDGE_UPPER_THRESH = 0.1   # defines the upper limit of the shared overlap between two entities to be called an edge intersection
EDGE_LOWER_THRESH = 0.01   # defines the lower limit of the shared overlap between two entities to be called an edge intersection

# calculate area
ca_poly_gdf["area"] = ca_poly_gdf["geometry"].area

# full join of all geometries
entity_join_all = gpd.overlay(
    ca_poly_gdf, 
    ca_poly_gdf,
    how = "intersection", keep_geom_type=False 
)

# remove self-intersections 
entity_join_all = entity_join_all[entity_join_all["entity_1"] != entity_join_all["entity_2"]]

# flag the types of intersections between organisations
# is org the same
entity_join_all["int_org_match"] = np.where(entity_join_all["organisation_entity_1"] == entity_join_all["organisation_entity_2"], True, False)

# the types of org-org matches
entity_join_all["int_org_types"] = np.select(
    [
        (entity_join_all["organisation_entity_1"] == 16) & (entity_join_all["organisation_entity_2"] == 16),
        (entity_join_all["organisation_entity_1"] != 16) & (entity_join_all["organisation_entity_2"] != 16),
        ((entity_join_all["organisation_entity_1"] != 16) & (entity_join_all["organisation_entity_2"] == 16)) |
        ((entity_join_all["organisation_entity_1"] == 16) & (entity_join_all["organisation_entity_2"] != 16))
    ],
    ["HE - HE", "LPA - LPA", "HE - other"],
    default = "-"
)

# does the entity entry date match?
entity_join_all["date_match"] = np.where(entity_join_all["entry_date_1"] == entity_join_all["entry_date_2"], True, False)

# calculate overlap %'s

entity_join_all["area_intersection"] = entity_join_all["geometry"].area

entity_join_all["p_pct_intersect"] = entity_join_all["area_intersection"] / entity_join_all["area_1"]
entity_join_all["pct_intersection"] = entity_join_all["area_intersection"] / (entity_join_all["area_1"] + entity_join_all["area_2"] - entity_join_all["area_intersection"])
entity_join_all["s_pct_intersect"] = entity_join_all["area_intersection"] / entity_join_all["area_2"]

# intersection area as % of smallest primary or secondary area
entity_join_all["pct_min_intersection"] = entity_join_all["area_intersection"] / entity_join_all[["area_1", "area_2"]].min(axis = 1)


entity_join_all["intersection_type"] = np.select(
    [
        (entity_join_all["p_pct_intersect"] >= MATCH_LOWER_THRESH) & (entity_join_all["s_pct_intersect"] >= MATCH_LOWER_THRESH),
        (entity_join_all["pct_min_intersection"] <= EDGE_UPPER_THRESH) & (entity_join_all["pct_min_intersection"] >= EDGE_LOWER_THRESH),
        (entity_join_all["pct_min_intersection"] < EDGE_LOWER_THRESH),
        ((entity_join_all["p_pct_intersect"] >= MATCH_LOWER_THRESH) | (entity_join_all["s_pct_intersect"] >= MATCH_LOWER_THRESH)),
        
    ],
    [
        "Complete match (two-way)", "Edge overlap", "Tiny edge - ignore", "Single match (one-way)"
    ],
    default = "Partial match"
)

print(len(entity_join_all))
# entity_join_all.head()

In [ ]:
# SUMMARISE OVERLAP ISSUES

# filter to just HE - LPA overlaps, and exclude tiny edges
ca_issues_he_lpa = entity_join_all[
    (entity_join_all["int_org_types"] == "HE - other") &
    (entity_join_all["intersection_type"] != "Tiny edge - ignore")
]

# group and count
ca_issues_he_lpa_count = ca_issues_he_lpa.groupby(
        ["entity_1"] # , "name_1", "reference_1", "organisation_entity_1", "geometry"
    ).agg(
        {"entity_2" : "count",
         "intersection_type" : lambda x: ', '.join(set(x))}
    ).reset_index(    
    )

# rename cols
ca_issues_he_lpa_count.rename(columns=
    {"entity_1":"entity", 
     "entity_2":"n_overlap_issues", 
     "intersection_type" : "overlap_issue_types"}, inplace = True)

# ca_issues_he_lpa_count.sort_values("n_overlap_issues", ascending = False)

In [ ]:
# create subset table with only single instances of each issue
overlap_issues_dist = ca_issues_he_lpa.copy()

overlap_issues_dist["entity_join"] = overlap_issues_dist.apply(lambda x: '-'.join(list(map(str, sorted(x[["entity_1", "entity_2"]])))), axis=1)

# extra sort to make sure matches to Historic England always show as Historic England as org 2 
overlap_issues_dist["name_for_sort"] = np.where(overlap_issues_dist["organisation_entity_1"] == 16, "Z", "A")
overlap_issues_dist.sort_values(["entity_join", "name_for_sort"], ascending=True, inplace=True)

overlap_issues_dist.drop_duplicates(subset="entity_join", inplace = True)  #Drop them by name

# flag entities with multiple issues
all_ents = pd.concat([overlap_issues_dist["entity_1"], overlap_issues_dist["entity_2"]], ignore_index = True)
multi_issue_ents = all_ents.loc[all_ents.duplicated()]

overlap_issues_dist["multi_issue_entities"] = np.where(
    (overlap_issues_dist["entity_2"].isin(multi_issue_ents)) |
    (overlap_issues_dist["entity_1"].isin(multi_issue_ents)),
    True, False)

# add in action field
overlap_issues_dist["action"] = np.select(
    [
        (overlap_issues_dist["intersection_type"] == "Complete match (two-way)") &
        (overlap_issues_dist["multi_issue_entities"] == False) 

    ],
    ["remap"],
    default = "investigate"
)

# write report to output dir
print(len(overlap_issues_dist))
# overlap_issues_dist.head()
overlap_issues_dist.sort_values(["organisation_name_1", "action"]
                                ).to_csv(os.path.join(output_dir, f"conservation_area_duplicates_report_{td}.csv"), index = False)

In [ ]:
# write redirections to output dir

redirections = overlap_issues_dist[overlap_issues_dist["action"] == "remap"]
n_redirects = len(redirections)

old_entity_update = pd.DataFrame({
    "old-entity" : redirections["entity_2"],
    "status" : ["301"] * n_redirects,
    "entity" : redirections["entity_1"],
    "end-date" : [""] * n_redirects,
    "notes" : ["redirect Historic England duplicate to LPA entity"] * n_redirects,
    "entry-date" : [td] * n_redirects,
    "start-date" : [""] * n_redirects
})

old_entity_update.to_csv(os.path.join(output_dir, f"redirections_old-entity-update_{td}.csv"), index = False)

### Combine (for mapping)

In [ ]:
# Join LPAs with intersecting CAs to CA geometry, and then to CA overlap issues

print(len(lpa_ca_join))

lpa_ca_all = lpa_ca_join[["lpa_reference", "lpa_name", "ODP_flag", "entity", "organisation_entity", "organisation_name"]].merge(
        ca_poly_gdf[["entity", "reference", "name", "geometry"]],
        how = "left",
        on = "entity"
    ).merge(
        ca_issues_he_lpa_count,
        how = "left",
        on = "entity"
    )

lpa_ca_all["overlap_issue_types"].replace(np.nan, "No overlap issues", inplace=True)
lpa_ca_all["has_overlap_issues"] = np.where(lpa_ca_all["n_overlap_issues"].notnull(), True, False)
lpa_ca_all = gpd.GeoDataFrame(lpa_ca_all, geometry="geometry")

print(len(lpa_ca_all))
# lpa_ca_all.head()

In [ ]:
ODP_overlap_issues_count = lpa_ca_all[
    (lpa_ca_all["has_overlap_issues"]) &
    (lpa_ca_all["ODP_flag"])].groupby("lpa_name").size().reset_index(name = "count").sort_values("count", ascending=False)

nODP_overlap_issues_count = lpa_ca_all[
    (lpa_ca_all["has_overlap_issues"]) &
    (~lpa_ca_all["ODP_flag"])].groupby("lpa_name").size().reset_index(name = "count").sort_values("count", ascending=False)

## Map
### ODP LPA issues
(see next section for non-ODP LPA issues)

In [ ]:
print("ODP LPAs by number of overlap issues")
ODP_overlap_issues_count

In [ ]:
ODP_dataset_options = dict(zip(ODP_overlap_issues_count["lpa_name"], ODP_overlap_issues_count["lpa_name"]))

ODP_dataset_dropdown = widgets.Dropdown(
    options=ODP_dataset_options,
    description="Select LPA:",
)

widgets.interact(display_lpa_issues, lpa_name=ODP_dataset_dropdown)

### non-ODP LPA issues

In [ ]:
print("non-ODP LPAs by number of overlap issues")
nODP_overlap_issues_count

In [ ]:
nODP_dataset_options = dict(zip(nODP_overlap_issues_count["lpa_name"], nODP_overlap_issues_count["lpa_name"]))

nODP_dataset_dropdown = widgets.Dropdown(
    options=nODP_dataset_options,
    description="Select LPA:",
)

widgets.interact(display_lpa_issues, lpa_name=nODP_dataset_dropdown)